In [1]:
!pip install torchgeo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.3/605.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.8/855.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torchgeo

import matplotlib.pyplot as plt


In [3]:
nir_img = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/2024350_Mosaik_NIR.tif"
rgb_img = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/2024350_Mosaik_RGB.tif"
mask_img = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/out_data/final_lulc_train_remapped_2.tif"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from torchgeo.datasets import IntersectionDataset
from torchgeo.datasets import RasterDataset, roi_split
from torchgeo.samplers import RandomGeoSampler
from torch.utils.data import DataLoader
from torchgeo.datasets import RasterDataset, IntersectionDataset, stack_samples




In [ ]:
import torchgeo
from torchgeo.datasets import (
    IntersectionDataset,
    RasterDataset,
    BoundingBox,
    stack_samples
)
from torchgeo.samplers import RandomGeoSampler
from torch.utils.data import DataLoader

# 1. Full Dataset
rgb_data = RasterDataset(paths=rgb_img, crs=31256)
nir_data = RasterDataset(paths=nir_img, crs=31256)
mask_dataset = RasterDataset(paths=mask_img, crs=31256)
mask_dataset.is_image = False
rgb_nir_data = IntersectionDataset(rgb_data, nir_data)
dataset = IntersectionDataset(rgb_nir_data, mask_dataset)

# 2. Full Region of Interest (ROI)
minx = -101193.4376
maxx = -25073.1548
miny = 172175.6434
maxy = 225951.9085
mint, maxt = 0, 0
custom_roi = BoundingBox(minx, maxx, miny, maxy, mint, maxt)

# --- NEW: Splitting the ROI for Train/Val/Test ---

# 1. Define split percentages and patch size/count
train_pct = 0.70
val_pct = 0.15
# test_pct is implicitly 0.15

patch_size = 512
train_samples = 5000 # Number of patches in one epoch
val_samples = 1000
test_samples = 1000
# train_samples = 50 # Number of patches in one epoch
# val_samples = 15
# test_samples = 15


# 2. Calculate the spatial split boundaries
# We'll split vertically along the x-axis.
width = custom_roi.maxx - custom_roi.minx
train_val_split_x = custom_roi.minx + width * train_pct
val_test_split_x = train_val_split_x + width * val_pct

# 3. Create three new, non-overlapping BoundingBox objects
train_roi = BoundingBox(
    minx=custom_roi.minx,
    maxx=train_val_split_x,
    miny=custom_roi.miny,
    maxy=custom_roi.maxy,
    mint=mint,
    maxt=maxt
)

val_roi = BoundingBox(
    minx=train_val_split_x,
    maxx=val_test_split_x,
    miny=custom_roi.miny,
    maxy=custom_roi.maxy,
    mint=mint,
    maxt=maxt
)

test_roi = BoundingBox(
    minx=val_test_split_x,
    maxx=custom_roi.maxx,
    miny=custom_roi.miny,
    maxy=custom_roi.maxy,
    mint=mint,
    maxt=maxt
)

print("--- Spatial Split ROIs ---")
print(f"Train ROI: {train_roi}")
print(f"Val ROI:   {val_roi}")
print(f"Test ROI:  {test_roi}")
print("--------------------------\n")


# 4. Create a unique Sampler and DataLoader for each split
# Note: They all use the SAME 'dataset' object, but their 'roi' restricts them.
batch_size = 4

# Training DataLoader
train_sampler = RandomGeoSampler(dataset, size=patch_size, length=train_samples, roi=train_roi)
train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size, collate_fn=stack_samples)

# Validation DataLoader
val_sampler = RandomGeoSampler(dataset, size=patch_size, length=val_samples, roi=val_roi)
val_dataloader = DataLoader(dataset, sampler=val_sampler, batch_size=batch_size, collate_fn=stack_samples)

# Testing DataLoader
test_sampler = RandomGeoSampler(dataset, size=patch_size, length=test_samples, roi=test_roi)
test_dataloader = DataLoader(dataset, sampler=test_sampler, batch_size=batch_size, collate_fn=stack_samples)

print("DataLoaders created successfully!")
print(f"Train Dataloader: {len(train_dataloader)} batches of size {batch_size}")
print(f"Val Dataloader:   {len(val_dataloader)} batches of size {batch_size}")
print(f"Test Dataloader:  {len(test_dataloader)} batches of size {batch_size}")



--- Spatial Split ROIs ---
Train ROI: BoundingBox(minx=-101193.4376, maxx=-47909.23964000001, miny=172175.6434, maxy=225951.9085, mint=0, maxt=0)
Val ROI:   BoundingBox(minx=-47909.23964000001, maxx=-36491.19722000001, miny=172175.6434, maxy=225951.9085, mint=0, maxt=0)
Test ROI:  BoundingBox(minx=-36491.19722000001, maxx=-25073.1548, miny=172175.6434, maxy=225951.9085, mint=0, maxt=0)
--------------------------

DataLoaders created successfully!
Train Dataloader: 1250 batches of size 4
Val Dataloader:   250 batches of size 4
Test Dataloader:  250 batches of size 4


In [10]:
import torch
import numpy as np
import os

In [13]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from torchmetrics.classification import MulticlassJaccardIndex
from tqdm import tqdm
import numpy as np

# --- 1. Configuration & Setup ---
# Assume train_dataloader, val_dataloader, test_dataloader are already created
# using the spatial splitting method from the previous step.

# Hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

# Hyperparameters
learning_rate = 1e-4
num_epochs = 20
batch_size = 4 # Reduced batch size to mitigate CUDA out of memory errors
loss_weight_focal = 2.0
loss_weight_dice = 1.0

# Class Configuration
# Your original masks have NoData=0 and valid classes 1-39.
# We need to map these for the model's output channels (which are 0-indexed).
# Valid classes: 1, 2, ..., 39 (39 total classes)
# Model output classes: 0, 1, ..., 38
num_actual_classes = 39
# We will map original NoData (0) to this special index so the loss function ignores it.
ignore_index_value = -1 # A common choice for ignore_index

# Create the remapping dictionary: {1:0, 2:1, ..., 39:38} and {0: -1}
# The remapping will be applied within the training/evaluation loop
# remapping_dict = {orig_id: new_id for orig_id, new_id in zip(range(1, num_actual_classes + 1), range(num_actual_classes))}
# remapping_dict[0] = ignore_index_value


# Model
model = smp.Segformer(
    encoder_name="mit_b3",
    encoder_weights="imagenet",
    in_channels=4,          # RGB + NIR
    classes=num_actual_classes, # Model outputs 39 channels, corresponding to classes 0-38
).to(device)
# Loss Functions
loss_focal = smp.losses.FocalLoss(mode="multiclass", ignore_index=ignore_index_value, gamma=2.0)
loss_dice = smp.losses.DiceLoss(mode="multiclass", ignore_index=ignore_index_value, from_logits=True)

# Combined Loss Criterion
def combined_loss(pred, target):
    focal = loss_focal(pred, target)
    dice = loss_dice(pred, target)
    return (loss_weight_focal * focal) + (loss_weight_dice * dice)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Metrics (mIoU)
# We use separate metric objects for train and val to keep their states separate.
train_met = MulticlassJaccardIndex(num_classes=num_actual_classes, ignore_index=ignore_index_value).to(device)
val_met = MulticlassJaccardIndex(num_classes=num_actual_classes, ignore_index=ignore_index_value).to(device)


# --- 3. Training & Evaluation Functions ---

# Helper function to remap masks - REMOVED as per user request
# def remap_masks(masks, ignore_val, num_classes):
#     """
#     Remaps original mask values (0 to num_classes) to model's expected range (0 to num_classes-1)
#     and the ignore value.
#     Original: 0 (NoData), 1..num_classes (Classes)
#     Remapped: ignore_val (NoData), 0..num_classes-1 (Classes)
#     """
#     remapped = torch.full_like(masks, ignore_val, dtype=torch.long)
#     # Remap classes 1 to num_classes to 0 to num_classes-1
#     for original_class in range(1, num_classes + 1):
#         remapped[masks == original_class] = original_class - 1
#     # Original 0 (NoData) remains ignore_val
#     return remapped


def train_one_epoch(model, criterion, optimizer, dataloader, device, metrics, ignore_index_value, num_actual_classes):
    model.train()
    metrics.reset()
    total_loss = 0.0

    for sample in tqdm(dataloader, desc="Training"):
        images = sample['image'].to(device).float()
        masks = sample['mask'].to(device) # Assume masks are already remapped or in the correct format
        # --- FIX: Squeeze the channel dimension from the mask ---
        if masks.dim() == 4 and masks.shape[1] == 1:
            masks = masks.squeeze(1)
        # ---------------------------------------------------------

        # Preprocessing
        images = images / 255.0 # IMPORTANT: Assuming 16-bit data. If 8-bit, use 255.0.
        # remapped_masks = remap_masks(masks, ignore_index_value, num_actual_classes) # REMOVED


        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks) # Use the original masks (assumed to be remapped)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update metrics and loss
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        # Ensure predictions are also remapped to the metric's expected range if needed
        # (though argmax output should be 0..num_classes-1, which matches the metric)
        metrics.update(preds, masks) # Use the original masks for metrics


    avg_loss = total_loss / len(dataloader)
    iou = metrics.compute()
    return avg_loss, iou

def evaluate(model, criterion, dataloader, device, metrics, ignore_index_value, num_actual_classes):
    model.eval()
    metrics.reset()
    total_loss = 0.0

    with torch.no_grad():
        for sample in tqdm(dataloader, desc="Evaluating"):
            images = sample['image'].to(device).float()
            masks = sample['mask'].to(device) # Assume masks are already remapped or in the correct format
            # --- FIX: Squeeze the channel dimension from the mask ---
            if masks.dim() == 4 and masks.shape[1] == 1:
                masks = masks.squeeze(1)
        # ---------------------------------------------------------
            images = images / 255.0 # Adjust for your data type
            # remapped_masks = remap_masks(masks, ignore_index_value, num_actual_classes) # REMOVED

            outputs = model(images)
            loss = criterion(outputs, masks) # Use the original masks (assumed to be remapped)

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            metrics.update(preds, masks) # Use the original masks for metrics


    avg_loss = total_loss / len(dataloader)
    iou = metrics.compute()
    return avg_loss, iou

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

In [ ]:
# --- 4. Main Training Loop ---
best_val_miou = 0.0

for epoch in range(num_epochs):
    print(f"--- Epoch {epoch+1}/{num_epochs} ---")
    model_save_path = os.path.join(r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/Model_training/DeeplabV3", f"best_model_{str(epoch+15)}.pth")

    # Train
    train_loss, train_miou = train_one_epoch(model, combined_loss, optimizer, train_dataloader, device, train_met, ignore_index_value, num_actual_classes)
    print(f"Train -> Loss: {train_loss:.4f}, mIoU: {train_miou:.4f}")

    # Validate
    val_loss, val_miou = evaluate(model, combined_loss, val_dataloader, device, val_met, ignore_index_value, num_actual_classes)
    print(f"Val   -> Loss: {val_loss:.4f}, mIoU: {val_miou:.4f}")

    # Save the best model based on validation mIoU
    if val_miou > best_val_miou:
        best_val_miou = val_miou
        torch.save(model.state_dict(), model_save_path)
        print(f"** New best model saved with mIoU: {best_val_miou:.4f} **\n")
    else:
        print("\n")
    torch.cuda.empty_cache()




# --- 5. Final Testing on Unseen Data ---
print("--- Training Finished ---")
print("Loading best model for final testing...")

# Load the best performing model
model.load_state_dict(torch.load(model_save_path))

# Create a new metric object for testing
test_met = MulticlassJaccardIndex(num_classes=num_actual_classes, ignore_index=ignore_index_value).to(device)

# Evaluate on the test set
test_loss, test_miou = evaluate(model, combined_loss, test_dataloader, device, test_met, ignore_index_value, num_actual_classes)

print("--- Final Test Results ---")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test mIoU: {test_miou:.4f}")